<a href="https://colab.research.google.com/github/Takumi173/JPMA2022TF1-1/blob/main/JPMA_2022_TF_1_1_demo_(4)%E3%82%A2%E3%83%B3%E3%82%B5%E3%83%B3%E3%83%96%E3%83%AB%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E4%BD%9C%E6%88%90%E3%81%A8%E6%A4%9C%E8%A8%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続

In [1]:
# データ受け渡しのためにGoogle Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# データ保存ディレクトリの指定
datadir = '/content/drive/MyDrive/datadir/'

Mounted at /content/drive


## データのロード

In [2]:
# 分かち書き済みのテキストとベクトル化したデータを読み込み
import pickle

with open(datadir + 'datadic.pkl', 'rb') as f:
  datadic = pickle.load(f)

## pip install

In [3]:
!pip install transformers
!pip install mecab-python3 fugashi 
!pip install jaconv neologdn
!pip install sentencepiece
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.8/577.8 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.9/615.9 KB 49.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 225.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jaconv: filename=jaconv-0.3.3-py3-none-any.whl size=16149 sha256=f332c6e194227635cf63102b12d90ce44b1325abf8d4a3e978985

## データロードと環境構築

In [4]:
import jaconv
import unicodedata
import neologdn
import re
import torch

# デバイス設定
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [5]:
# MeCabとNEologdの設定
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y

# 環境変数でmecabrcの場所を指定
import os
os.environ['MECABRC'] = "/etc/mecabrc" 

# NEologdの展開場所を取得
import subprocess
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# 万病辞書のダウンロードと設定
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic
manbyo_dic_path = 'MANBYO_201907_Dic-utf8.dic'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 20 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-up

## テストデータの準備

In [31]:
from sklearn.model_selection import train_test_split

def tts(DataSet, Mode):
  train, test = train_test_split(DataSet, test_size=0.15, random_state = 0)
  if Mode == 'train':
    r = train  # 学習&検証に使用した85%のデータを返す
  elif Mode == 'test':
    r = test  # モデルの学習＆検証に使用しなかった15%を返す

  return r

# アンサンブルモデルの学習/検証モード指定（Appendixの実行時に切り替えが必要）
# 各モデルの学習＆検証に使用したデータを参照する
#Mode = 'train'

# 各モデルの学習＆検証に使用していないデータを参照する
Mode = 'test' 

# 各モデルからPrediction&Probabilityを取得

## sklearn Grid Search Models

In [32]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

# 検証対象となるラベル等を取得
label = tts(datadic['flg'], Mode)
texto = tts(datadic['original'], Mode)
textw = tts(datadic['wakati'], Mode)

# 結果格納用のDataFrameを定義
ensembles = pd.DataFrame(label, columns = ["Label"])
ensembles["Original Text"] = texto
ensembles["Wakati Text"] = textw
ensembles["Negative score"] = 0
ensembles["Positive score"] = 0

# Grid Searchでモデルを作成したベクトルデータ名を指定
vecs = [
    'BoW',
    'TFIDF',
    'Emb',
    'Emb_tfidf',
]

# 設定ベクトルに対してループ処理
for vecname in tqdm(vecs, total = len(vecs)):
  X_test = tts(datadic[vecname], Mode) # 対象ベクトルのデータを取得として分割
  pred_prob = 0                        # Probabilityの初期値を0に戻す

  # Grid Search結果の読み込み
  with open(datadir + vecname + '.pkl', 'rb') as f:
    gsmodels = pickle.load(f)

  # Grid Searchを実施した検索対象モデルに対してループ処理
  for clf in tqdm(gsmodels.keys(), total = len(gsmodels.keys())):
    if clf.endswith('_score'):
      # Scoreの結果は使用しないので無視する
      continue

    else:
      # Grid Search結果の最良モデルを読み込み
      bestclfs = gsmodels[clf].best_estimator_

      # Probabilityを取得（predict_probaを持たないモデルは算出）
      if clf in ['RID', 'LSVC']:
        d = bestclfs.decision_function(X_test)
        prob1 = np.exp(d) / np.sum(np.exp(d))
        prob0 = 1 - prob1
        pred_prob += np.stack([prob0,prob1],0).T
      else:
        pred_prob += bestclfs.predict_proba(X_test)

      # PredictionとProbabilityをDataFrameに格納
      predict = pd.DataFrame(bestclfs.predict(X_test), columns=[vecname + '+' + clf])
      probs = pd.DataFrame(pred_prob, columns=[vecname + '+' + clf + '_Neg', vecname + '+' + clf + '_Pos'])

      # 出力用DataFrameに結合
      ensembles = pd.concat([ensembles, predict], axis = 1) 
      ensembles = pd.concat([ensembles, probs], axis = 1)
      
      # Probabilityの合計値を算出
      ensembles["Negative score"] += probs[vecname + '+' + clf + '_Neg']
      ensembles["Positive score"] += probs[vecname + '+' + clf + '_Pos'] 

# 結果の確認
display(ensembles)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

,Label,Original Text,Wakati Text,Negative score,Positive score,BoW+LR,BoW+LR_Neg,BoW+LR_Pos,BoW+RID,BoW+RID_Neg,...,Emb_tfidf+MLP2_Pos,Emb_tfidf+MLP3,Emb_tfidf+MLP3_Neg,Emb_tfidf+MLP3_Pos,Emb_tfidf+AB1,Emb_tfidf+AB1_Neg,Emb_tfidf+AB1_Pos,Emb_tfidf+AB2,Emb_tfidf+AB2_Neg,Emb_tfidf+AB2_Pos
0,1,2022/01/27、SARS-CoV-2検査(PCR)で、COVID-19陽性を示した。,0 0 0 SARS CoV 0 検査 PCR だ COVID 0 陽性 示す た,150.481110,213.518890,1,0.018447,0.981553,1,1.016468,...,6.550124,1,3.533109,7.466891,1,3.700384,8.299616,1,3.982860,9.017140
1,0,さらなる以上の追加情報は期待できない。,以上 追加 情報 期待 できる ない,344.716577,19.283423,0,0.983039,0.016961,0,1.982539,...,0.711499,0,10.213586,0.786414,0,11.212497,0.787503,0,12.048738,0.951262
2,0,脳神経系、運動系、感覚系に異常はなく、四肢腱反射正常で病的反射は両側陰性だった。,脳神経 系 運動系 感覚 系 異常 ない 四肢腱反射正常 だ 病的 反射 両側 陰性 だ た,257.929276,106.070724,0,0.999115,0.000885,0,1.998844,...,5.056857,0,5.576586,5.423414,1,5.577972,6.422028,1,5.836768,7.163232
3,1,2021/12/17(ワクチン接種15日後)、症状の増悪があり入院となった。,0 0 0 ワクチン接種 0 日 後 症状 増悪 ある 入院 なる た,125.300384,238.699616,1,0.001143,0.998857,1,0.999019,...,6.690185,1,3.333050,7.666950,1,3.366426,8.633574,1,3.573455,9.426545
4,0,患者の家族歴は特になしと報告された。,患者 家族 歴 ない 報告 する れる た,324.127667,39.872333,0,0.998441,0.001559,0,1.998107,...,3.398679,0,7.199966,3.800034,0,8.030258,3.969742,0,8.703052,4.296948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,1,両側の肺炎と呼吸不全を指摘され入院。,両側 肺炎 呼吸不全 指摘 する れる 入院,101.873342,262.126658,1,0.000027,0.999973,1,0.994413,...,7.850446,1,2.149554,8.850446,1,2.149933,9.850067,1,2.251106,10.748894
749,0,その他の画像検査は実施されなかった。,その他 画像 検査 実施 する れる ない た,351.909405,12.090595,0,0.999952,0.000048,0,1.999709,...,0.517383,0,10.469788,0.530212,0,11.469573,0.530427,0,12.333335,0.666665
750,0,2022/05/24、事象の転帰は軽快であった(報告の通り)。,0 0 0 事象 転帰 軽快 だ ある た 報告 通り,332.758848,31.241152,0,0.997844,0.002156,0,1.997492,...,0.316341,0,10.683659,0.316341,0,11.683154,0.316846,0,12.566897,0.433103
751,0,2022/03/07症状軽快し、退院。,0 0 0 症状 軽快 する 退院,331.859960,32.140040,0,0.999429,0.000571,0,1.999162,...,0.797815,0,10.202185,0.797815,0,11.202155,0.797845,0,12.079503,0.920497


## BERT Models

In [33]:
# 評価関数の定義
def eval(eval_dataloader, model, device, tqdm):
  logits     = []
  inputs     = []

  model.eval()    # 検証モード
  for n_iter, d in tqdm(enumerate(eval_dataloader), total=len(eval_dataloader)):
    with torch.no_grad():
      outputs = model(
        d[0].to(device),                    # input_ids_t
        attention_mask = d[1].to(device),   # attention_masks_t
        token_type_ids=None
        )
    
    logits += outputs.logits.sigmoid().cpu().tolist()
    inputs += d[0].tolist()
  
  eval_res = pd.DataFrame(logits, columns=['logit0', 'logit1'])
  eval_pred = np.argmax(eval_res.values, axis=1).tolist()
  eval_res['pred']  = eval_pred
  eval_res['input_ids']  = inputs

  return eval_res

In [34]:
from transformers import  BertJapaneseTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader, TensorDataset

# BERT Prediction関数の定義
def bertprediction(model_name, text, label):
  # 事前学習モデルの読み込み
  model = BertForSequenceClassification.from_pretrained(
      model_name,                    # 読み込むモデル名
      num_labels = 2,                # Binary classification
      output_attentions = False,     # Attentionの出力
      output_hidden_states = False,  # 隠れ層の出力
      )
  model.to(device)

  # 事前学習モデルのトークナイザ設定
  MeCabDic = {"mecab_dic": None, "mecab_option": "-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path}

  tokenizer = BertJapaneseTokenizer.from_pretrained(
      model_name,
      word_tokenizer_type = "mecab",
      mecab_kwargs = MeCabDic
      )

  # 最大トークン数の設定
  # Fine-tuningしたBERTモデルの最大長に合わせる
  model_seq_len = 512

  # テストデータが最大長に満たない場合は計算量削減のためテストデータの最大長に切り落とし
  max_tk = 0
  for i, chktoken in enumerate(text):
    tk = tokenizer.tokenize(chktoken)
    if len(tk) > max_tk:
      max_tk = len(tk)
      id = i

  max_len = max_tk + 2 if max_tk + 2 < model_seq_len else model_seq_len

  # 最大長データの確認
  tokchk = tokenizer.encode_plus(
      text[id],
      add_special_tokens = True,        # スペシャルトークンの追加
      truncation = True,                # モデル定義長を超える場合の切り捨て
      max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
      padding = 'max_length',           # 最大長までPADDING
      return_overflowing_tokens = True, # 切り捨てられたトークンを返す
      num_truncated_tokens = True       # 切り捨てられたトークン数を返す
      )

  print("最大トークン数:", max_tk)
  print("*** 最大トークン数に分割されるテキスト ***")
  print("  ", text[id])
  print("*** BERTに入力されるテキスト ***")
  print("  ", tokenizer.decode(tokchk['input_ids']))
  print("*** 切り捨てられるテキスト ***")
  print("  ", tokenizer.decode(tokchk['overflowing_tokens']))

  # トークナイズ処理
  # 必要なToken IDとAttentionマスクを取得
  token_ids = []
  attention_masks = []

  for t in text:
    tknzd = tokenizer.encode_plus(
        t,
        add_special_tokens = True,        # スペシャルトークンの追加
        truncation = True,                # モデル定義長を超える場合の切り捨て
        max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
        padding = 'max_length'            # 最大長までPADDING
        )
    token_ids.append(tknzd['input_ids'])
    attention_masks.append(tknzd['attention_mask'])

  # tensor型に変換
  token_ids_t = torch.tensor(token_ids)
  attention_masks_t = torch.tensor(attention_masks)

  # データセットとデータローダーの作成
  eval_dataset = TensorDataset(token_ids_t, attention_masks_t)
  eval_dataloader = DataLoader(
      eval_dataset,
      batch_size = 128,
      shuffle = False,
      drop_last = False
      )

  # Predictionの実行
  out = eval(eval_dataloader, model, device, tqdm)

  out['Text'] = [t.strip('[CLS] [SEP] [PAD]') for t in tokenizer.batch_decode(out['input_ids'])]
  out['text1'] = text
  labeldf = pd.DataFrame(label, columns = ["Label"])

  out = pd.concat([labeldf, out], axis = 1 )

  return out

In [35]:
# BERTのモデル名とFine-tuning済みのモデルパスを指定
models = [
  ('BERT+BERT0', datadir + 'bert/BERT_MODEL_0'),
  ('BERT+BERT1', datadir + 'bert/BERT_MODEL_1'),
  ('BERT+BERT2', datadir + 'bert/BERT_MODEL_2'),
  ('BERT+BERT3', datadir + 'bert/BERT_MODEL_3'),
  ('BERT+BERT4', datadir + 'bert/BERT_MODEL_4'),
]

# BERTモデルのPredictionとProbabilityを取得
for name, model in tqdm(models, total= len(models)):
  df = bertprediction(model, textw, label)

  df = df.rename(columns={'pred': name, 'logit0': name + '_Neg', 'logit1': name + '_Pos'})

  ensembles = pd.concat([ensembles, df[name]], axis = 1)
  ensembles = pd.concat([ensembles, df[name + '_Neg']], axis = 1)
  ensembles = pd.concat([ensembles, df[name + '_Pos']], axis = 1)
  ensembles["Negative score"] += df[name + '_Neg']
  ensembles["Positive score"] += df[name + '_Pos']


  0%|          | 0/5 [00:00<?, ?it/s]

Keyword arguments {'num_truncated_tokens': True} not recognized.


最大トークン数: 105
*** 最大トークン数に分割されるテキスト ***
   血栓形成傾向 血小板減少 の 原因 病態 特発性 血小板減少 性 紫斑病 抗リン脂質抗体症候群 血栓性 微小血管症 考える た 各種 自己抗体 骨髄像 検査 結果 いずれ 疾患 典型 的 だ ない ため 経過 Vaccine Induced Immune Thrombocytopenia AND Thrombosis VITT 可能性 考える 免疫グロブリン 静 療法 0g kg day 0 日間 実施 する た
*** BERTに入力されるテキスト ***
   [CLS] 血栓形成傾向 血小板減少 の 原因 病態 特発性 血小板減少 性 紫斑病 抗リン脂質抗体症候群 血栓性 微小血管症 考える た 各種 自己抗体 骨髄像 検査 結果 いずれ 疾患 典型 的 だ ない ため 経過 Vaccine Induced Immune Thrombocytopenia AND Thrombosis VITT 可能性 考える 免疫グロブリン 静 療法 0g kg day 0 日間 実施 する た [SEP]
*** 切り捨てられるテキスト ***
   


  0%|          | 0/6 [00:00<?, ?it/s]

Keyword arguments {'num_truncated_tokens': True} not recognized.


...

最大トークン数: 105
*** 最大トークン数に分割されるテキスト ***
   血栓形成傾向 血小板減少 の 原因 病態 特発性 血小板減少 性 紫斑病 抗リン脂質抗体症候群 血栓性 微小血管症 考える た 各種 自己抗体 骨髄像 検査 結果 いずれ 疾患 典型 的 だ ない ため 経過 Vaccine Induced Immune Thrombocytopenia AND Thrombosis VITT 可能性 考える 免疫グロブリン 静 療法 0g kg day 0 日間 実施 する た
*** BERTに入力されるテキスト ***
   [CLS] 血栓形成傾向 血小板減少 の 原因 病態 特発性 血小板減少 性 紫斑病 抗リン脂質抗体症候群 血栓性 微小血管症 考える た 各種 自己抗体 骨髄像 検査 結果 いずれ 疾患 典型 的 だ ない ため 経過 Vaccine Induced Immune Thrombocytopenia AND Thrombosis VITT 可能性 考える 免疫グロブリン 静 療法 0g kg day 0 日間 実施 する た [SEP]
*** 切り捨てられるテキスト ***
   


  0%|          | 0/6 [00:00<?, ?it/s]

Keyword arguments {'num_truncated_tokens': True} not recognized.


最大トークン数: 105
*** 最大トークン数に分割されるテキスト ***
   血栓形成傾向 血小板減少 の 原因 病態 特発性 血小板減少 性 紫斑病 抗リン脂質抗体症候群 血栓性 微小血管症 考える た 各種 自己抗体 骨髄像 検査 結果 いずれ 疾患 典型 的 だ ない ため 経過 Vaccine Induced Immune Thrombocytopenia AND Thrombosis VITT 可能性 考える 免疫グロブリン 静 療法 0g kg day 0 日間 実施 する た
*** BERTに入力されるテキスト ***
   [CLS] 血栓形成傾向 血小板減少 の 原因 病態 特発性 血小板減少 性 紫斑病 抗リン脂質抗体症候群 血栓性 微小血管症 考える た 各種 自己抗体 骨髄像 検査 結果 いずれ 疾患 典型 的 だ ない ため 経過 Vaccine Induced Immune Thrombocytopenia AND Thrombosis VITT 可能性 考える 免疫グロブリン 静 療法 0g kg day 0 日間 実施 する た [SEP]
*** 切り捨てられるテキスト ***
   


  0%|          | 0/6 [00:00<?, ?it/s]

## 各モデルの評価スコアを計算

In [36]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# 実行結果を保存したensemblesを読みこんで正解ラベルと各Predictionを比較する
x = []
for c in ensembles.columns[5:]:
  if not c.endswith(('_Pos','_Neg')):
    y = []
    y.append(str(c).split('+')[0])
    try:
      y.append(str(c).split('+')[1])
    except:
      y.append('')

    y.append(f1_score(ensembles["Label"], ensembles[c].astype(int)))
    y.append(accuracy_score(ensembles["Label"], ensembles[c].astype(int)))
    y.append(recall_score(ensembles["Label"], ensembles[c].astype(int)))
    y.append(precision_score(ensembles["Label"], ensembles[c].astype(int)))
    x.append(y)

  else:
    continue

scores = pd.DataFrame(x, columns = ["Vector", "Method", "F1", "Accuracy", "Recall", "Precision"])
scores["idx"] = scores["Vector"]+"+"+scores["Method"]
scores.set_index("idx",inplace=True)

# 結果と評価スコアをCSVで保存
ensembles.to_csv(datadir + 'ensembles_' + Mode + '.csv')
scores.to_csv(datadir + 'scores_' + Mode + '.csv')


# アンサンブル

## アンサンブルモデルの選択

In [37]:
import re

# Gred searchで分割検索したモデルがあるため、モデル名から分割数字を削除
selection = scores.copy()
selection["Method_"] = selection["Method"].apply(lambda x: re.sub("[0-9]","",x))

# グループ化で最大スコアのみ抽出する
max_idx = selection.groupby(["Vector","Method_"])["F1"].idxmax()
selection = selection[selection.index.isin(max_idx)]

# 複合キーをIndexに再設定
selection["idx"] = selection["Vector"]+"+"+selection["Method_"]
selection.set_index("idx",inplace=True)


<ipython-input-37-ce48a25b73e7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selection["idx"] = selection["Vector"]+"+"+selection["Method_"]


In [38]:
import pulp

# 以下のように最適なモデルを選択する
#     目的：選択したモデルの合計F1スコアが最も高くなるようにする
#   制約 1：各Vectorごとにスコアの良い手法を1つ選択する
#   制約 2：手法は重複しないように選択する

# 最適化問題の作成
problem = pulp.LpProblem('BestModels', pulp.LpMaximize)

# 評価に使用したVectorと手法のリスト
V = set(selection["Vector"])
M = set(selection["Method_"])

# 最適化変数の定義
x = pulp.LpVariable.dicts('x', [(v,m) for v in V for m in M], cat='Binary')

# 係数とするF1スコアの取得
def F1values(Tgt):
  v = Tgt[0]
  m = Tgt[1]
  try:
    x = selection.at[v+"+"+m,"F1"]
  except:
    x = 0
  return x

f1 = {}
for i in x.keys():
  f1[i] = F1values(i)

# 目的関数の定義
problem += pulp.lpSum( [f1[i] * x[i] for i in x.keys()] )   # F1スコアを係数として各Vectorと手法の組み合わせを0/1変数とした数式を定義

# 制約の定義
for v in V:
  problem += pulp.lpSum([x[v,m] for m in M]) == 1  # 各Vectorから1つずつ選択されること

for m in M:
  problem += pulp.lpSum([x[v,m] for v in V]) <= 1  # 選択された手法が重複しないこと

# 最適化Statusの確認
result = problem.solve()
print('Resolution status:', pulp.LpStatus[result])

# 最適な組み合わせの出力
result = {}
for v in V:
  result[v] = [m for m in M if x[v,m].value()==1]

# 選択されたモデルのスコアを確認
selected = []
for v in result.keys():
  selected.append(v + "+" + selection.at[v + "+" + result[v][0], "Method"])

print('*** 最適な組み合わせ ***')
display(scores[scores.index.isin(selected)])

# 選択されたモデル名を保存
with open(datadir + 'ensemble.pkl', 'wb') as f:
  pickle.dump(selected, f)

Resolution status: Optimal
*** 最適な組み合わせ ***


,Vector,Method,F1,Accuracy,Recall,Precision
idx,,,,,,
BoW+LR,BoW,LR,0.909091,0.911023,0.893333,0.925414
TFIDF+SVC,TFIDF,SVC,0.927536,0.926959,0.938667,0.916667
Emb+LSVC,Emb,LSVC,0.896366,0.897742,0.888000,0.904891
Emb_tfidf+MLP1,Emb_tfidf,MLP1,0.867532,0.864542,0.890667,0.845570
BERT+BERT3,BERT,BERT3,0.931174,0.932271,0.920000,0.942623


## 選択モデルの結果抽出とアンサンブル

In [39]:
import pandas as pd
#mode= 'train'
#mode= 'valid'
ensembles = pd.read_csv(datadir + 'ensembles_' + Mode + '.csv')


In [40]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# 必要なカラムを選択
cols_h = [
  "Label",
  "Original Text",
  "Wakati Text", 
  ]

cols_d = []
for v in result.keys():
  cols_d.append(v + "+" + selection.at[v + "+" + result[v][0], "Method"])

ensembles_d = ensembles[cols_h+cols_d]

# いずれか一つのモデルがPositiveと判定している場合はPositiveとする
vres = pd.DataFrame(ensembles_d[cols_d].sum(axis=1), columns=["ens_v"])
ensembles_d = pd.concat([ensembles_d, vres], axis = 1) 
ensembles_d["ens_v"] = ensembles_d["ens_v"].apply(lambda x: 0 if x == 0 else 1)

# 結果の確認
display(ensembles_d)

print('****: Ensemble Scores')
print('        F1:', f1_score(ensembles_d["Label"], ensembles_d["ens_v"]))
print('    Recall:', recall_score(ensembles_d["Label"], ensembles_d["ens_v"]))
print('  Accuracy:', accuracy_score(ensembles_d["Label"], ensembles_d["ens_v"]))
print(' Precision:', precision_score(ensembles_d["Label"], ensembles_d["ens_v"]))

,Label,Original Text,Wakati Text,Emb_tfidf+MLP1,TFIDF+SVC,Emb+LSVC,BERT+BERT3,BoW+LR,ens_v
0,1,2022/01/27、SARS-CoV-2検査(PCR)で、COVID-19陽性を示した。,0 0 0 SARS CoV 0 検査 PCR だ COVID 0 陽性 示す た,1,1,1,1,1,1
1,0,さらなる以上の追加情報は期待できない。,以上 追加 情報 期待 できる ない,0,0,0,0,0,0
2,0,脳神経系、運動系、感覚系に異常はなく、四肢腱反射正常で病的反射は両側陰性だった。,脳神経 系 運動系 感覚 系 異常 ない 四肢腱反射正常 だ 病的 反射 両側 陰性 だ た,0,0,0,0,0,0
3,1,2021/12/17(ワクチン接種15日後)、症状の増悪があり入院となった。,0 0 0 ワクチン接種 0 日 後 症状 増悪 ある 入院 なる た,1,1,1,1,1,1
4,0,患者の家族歴は特になしと報告された。,患者 家族 歴 ない 報告 する れる た,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
748,1,両側の肺炎と呼吸不全を指摘され入院。,両側 肺炎 呼吸不全 指摘 する れる 入院,1,1,1,1,1,1
749,0,その他の画像検査は実施されなかった。,その他 画像 検査 実施 する れる ない た,0,0,0,0,0,0
750,0,2022/05/24、事象の転帰は軽快であった(報告の通り)。,0 0 0 事象 転帰 軽快 だ ある た 報告 通り,0,0,0,0,0,0
751,0,2022/03/07症状軽快し、退院。,0 0 0 症状 軽快 する 退院,0,0,0,0,0,0


****: Ensemble Scores
        F1: 0.8757396449704142
    Recall: 0.9866666666666667
  Accuracy: 0.8605577689243028
 Precision: 0.7872340425531915


# Appendix: Stacking

## Stacking

In [41]:
# 選択モデルのProbability値を用いた新たなモデルを構築する
# タスクの目的を考慮するとRecall値を重視した上記アンサンブルモデルが良いと思われる
# 実行せずとも本編に影響なし
# 実行には「テストデータの準備」セクションで train モードを選択し、モデル学習に使用したデータのProbabilityを用いたPredictionモデルを事前作成する必要がある

import pandas as pd
from sklearn.linear_model import RidgeClassifier
import pickle

#Mode ='train'
#Mode ='test'
predscores = pd.read_csv(datadir + 'ensembles_' + Mode + '.csv')

with open(datadir + 'ensemble.pkl', 'rb') as f:
  selected = pickle.load(f)
print("アンサンブルで使用するモデル",selected)

# Labelと使用するモデルを指定
kcol = tuple(["Label"] + selected)

predscores_d = predscores
for c in predscores.columns:
  if c.startswith(kcol) == False:
    predscores_d = predscores_d.drop(columns = [c])

predscores_d_label = predscores_d["Label"]
predscores_d_logit = predscores_d.drop(columns = ["Label"])
for c in predscores_d_logit.columns:
  if c.endswith(('_Pos','_Neg')) == False:
    predscores_d_logit = predscores_d_logit.drop(columns = [c])

# Trainモードの場合はリッジ回帰のモデルを作成して保存する
if Mode == 'train':
  print('*** Train mode execution ***')
  ridge = RidgeClassifier(alpha=10).fit(predscores_d_logit, predscores_d_label)

  sres = pd.DataFrame(ridge.predict(predscores_d_logit), columns=["ens_s"])
  ensembles_d = pd.concat([ensembles_d, sres], axis = 1) 

  with open(datadir + 'logitensembles.pkl', 'wb') as f:
    pickle.dump(ridge, f)

# Testモードの場合は保存したモデルを読み込んで予測結果を返す
elif Mode == 'test':
  print('*** Test mode execution ***')
  with open(datadir + 'logitensembles.pkl', 'rb') as f:
    ridge = pickle.load(f)

  sres = pd.DataFrame(ridge.predict(predscores_d_logit), columns=["ens_s"])
  ensembles_d = pd.concat([ensembles_d, sres], axis = 1) 


アンサンブルで使用するモデル ['Emb_tfidf+MLP1', 'TFIDF+SVC', 'Emb+LSVC', 'BERT+BERT3', 'BoW+LR']
*** Test mode execution ***


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- BERT+BERT3_Neg
- BERT+BERT3_Pos
- BoW+LR_Neg
- BoW+LR_Pos
- Emb+LSVC_Neg
- ...
Feature names seen at fit time, yet now missing:
- BERT+BERT2_Neg
- BERT+BERT2_Pos
- BoW+MLP2_Neg
- BoW+MLP2_Pos
- Emb+LGB_Neg
- ...

  warnings.warn(message, FutureWarning)


In [42]:
# Predictionスコアの確認
from sklearn.metrics import f1_score, accuracy_score, recall_score
print('****: vote scores')
print('      f1:', f1_score(ensembles_d["Label"], ensembles_d["ens_v"]))
print('  recall:', recall_score(ensembles_d["Label"], ensembles_d["ens_v"]))
print('\n****: stacking scores')
print('      f1:', f1_score(ensembles_d["Label"], ensembles_d["ens_s"]))
print('  recall:',recall_score(ensembles_d["Label"], ensembles_d["ens_s"]))


ensembles_d.to_csv(datadir + 'ensembles_d_' + Mode + '.csv')
predscores_d.to_csv(datadir + 'predscores_d_' + Mode + '.csv')

****: vote scores
      f1: 0.8757396449704142
  recall: 0.9866666666666667

****: stacking scores
      f1: 0.926892950391645
  recall: 0.9466666666666667


## Grid search

In [43]:
# Stackingモデルを作成するためのGrid Searchを試す

import pandas as pd
import pickle

#Mode ='train'
#Mode ='test'

with open(datadir + 'ensemble.pkl', 'rb') as f:
  selected = pickle.load(f)
print("アンサンブルで使用するモデル",selected)

ensembles = pd.read_csv(datadir + 'ensembles_' + Mode + '.csv')

# Labelと使用するモデルを指定
kcol = tuple(["Label"] + selected)

predscores_d = ensembles

for c in ensembles.columns:
  if c.startswith(kcol) == False:
    predscores_d = predscores_d.drop(columns = [c])
predscores_d

アンサンブルで使用するモデル ['Emb_tfidf+MLP1', 'TFIDF+SVC', 'Emb+LSVC', 'BERT+BERT3', 'BoW+LR']


,Label,BoW+LR,BoW+LR_Neg,BoW+LR_Pos,TFIDF+SVC,TFIDF+SVC_Neg,TFIDF+SVC_Pos,Emb+LSVC,Emb+LSVC_Neg,Emb+LSVC_Pos,Emb_tfidf+MLP1,Emb_tfidf+MLP1_Neg,Emb_tfidf+MLP1_Pos,BERT+BERT3,BERT+BERT3_Neg,BERT+BERT3_Pos
0,1,1,0.018447,0.981553,1,2.127831,1.872169,1,2.366988,0.633012,1,3.370599,5.629401,1,0.319220,0.730997
1,0,0,0.983039,0.016961,0,3.956042,0.043958,0,2.993478,0.006522,0,8.299732,0.700268,0,0.914637,0.100515
2,0,0,0.999115,0.000885,0,3.861765,0.138235,0,2.909108,0.090892,0,4.164585,4.835415,0,0.791990,0.142182
3,1,1,0.001143,0.998857,1,2.007946,1.992054,1,2.056436,0.943564,1,3.241033,5.758967,1,0.125330,0.857661
4,0,0,0.998441,0.001559,0,3.995895,0.004105,0,2.989436,0.010564,0,5.749341,3.250659,0,0.859938,0.142455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,1,1,0.000027,0.999973,1,1.988685,2.011315,1,1.989932,1.010068,1,2.149554,6.850446,1,0.106267,0.895472
749,0,0,0.999952,0.000048,0,3.999231,0.000769,0,2.998229,0.001771,0,8.485611,0.514389,0,0.916880,0.093684
750,0,0,0.997844,0.002156,0,3.973332,0.026668,0,2.929924,0.070076,0,8.683659,0.316341,0,0.862546,0.161610
751,0,0,0.999429,0.000571,0,3.997453,0.002547,0,2.855200,0.144800,0,8.202185,0.797815,0,0.888701,0.118721


In [44]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import LinearSVC

# 線形回帰モデルの Gred search を実行

clf_LR = LogisticRegression(max_iter=50000)
clf_RID = RidgeClassifier(max_iter=50000)
clf_LSVC = LinearSVC(max_iter=50000)

param_LR = {
    'C': [10**i for i in range(-5,5)]
     }

param_RID = {
    'alpha':  [10**i for i in range(-5,5)]
     }

param_LSVC = {
    'C': [10**i for i in range(-5,5)]
     }

clfs = {
    "LR": [clf_LR, param_LR],
    "RID": [clf_RID, param_RID],
    "LSVC": [clf_LSVC, param_LSVC], 
  }

def grid_search(clf, param, Data, Label):
  grid_search = GridSearchCV(
      clf,
      param,
      cv=5,
      scoring='f1') # f1でスコアリング
  return grid_search.fit(Data, Label)

predscores_d_label = predscores_d["Label"]
predscores_d_logit = predscores_d.drop(columns = ["Label"])
for c in predscores_d_logit.columns:
  if c.endswith(('_Pos','_Neg')) == False:
    predscores_d_logit = predscores_d_logit.drop(columns = [c])


X_train, X_test, y_train, y_test = train_test_split(predscores_d_logit, predscores_d_label, test_size=0.15, random_state = 0)

for clfname in clfs.keys():
  gs = grid_search(clfs[clfname][0], clfs[clfname][1], X_train, y_train)

  print('****', clfname)
  print(gs.score(X_test, y_test))
  display(pd.DataFrame(gs.cv_results_))


**** LR
0.9357798165137614


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004354,0.000801,0.001876,0.000257,0.00001,{'C': 1e-05},0.693878,0.707071,0.867257,0.857143,0.854545,0.795979,0.078206,10
1,0.005728,0.001272,0.001834,0.000105,0.0001,{'C': 0.0001},0.870229,0.861789,0.913386,0.866142,0.934426,0.889194,0.029235,9
2,0.005472,0.000307,0.001752,0.000106,0.001,{'C': 0.001},0.887218,0.888889,0.913386,0.875000,0.943089,0.901516,0.024233,8
3,0.006632,0.000303,0.001695,0.000007,0.01,{'C': 0.01},0.916031,0.890625,0.935484,0.890625,0.951613,0.916875,0.024215,7
4,0.009444,0.001721,0.001769,0.000121,0.1,{'C': 0.1},0.938462,0.913386,0.951613,0.914729,0.960630,0.935764,0.019079,6
5,0.016375,0.001668,0.001979,0.000412,1,{'C': 1},0.930233,0.936508,0.959350,0.930233,0.976378,0.946540,0.018368,1
6,0.017717,0.002479,0.001941,0.000166,10,{'C': 10},0.923077,0.936508,0.959350,0.930233,0.976378,0.945109,0.019805,2
7,0.013740,0.002223,0.001781,0.000111,100,{'C': 100},0.916031,0.936508,0.959350,0.920635,0.976378,0.941780,0.022994,3
8,0.013123,0.000918,0.001722,0.000040,1000,{'C': 1000},0.916031,0.936508,0.951613,0.920635,0.976378,0.940233,0.021999,4
9,0.014943,0.004001,0.002030,0.000467,10000,{'C': 10000},0.916031,0.936508,0.951613,0.920635,0.976378,0.940233,0.021999,4


**** RID
0.9549549549549551


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003127,0.001066,0.002094,0.000598,0.00001,{'alpha': 1e-05},0.917293,0.929134,0.959350,0.904762,0.976378,0.937383,0.026604,3
1,0.002497,0.000066,0.001818,0.000056,0.0001,{'alpha': 0.0001},0.917293,0.929134,0.959350,0.904762,0.976378,0.937383,0.026604,3
2,0.002467,0.000018,0.001793,0.000038,0.001,{'alpha': 0.001},0.917293,0.929134,0.959350,0.904762,0.976378,0.937383,0.026604,3
3,0.002633,0.000430,0.001943,0.000355,0.01,{'alpha': 0.01},0.917293,0.929134,0.959350,0.904762,0.976378,0.937383,0.026604,3
4,0.002771,0.000383,0.001999,0.000312,0.1,{'alpha': 0.1},0.917293,0.929134,0.959350,0.912000,0.968750,0.937305,0.022726,7
5,0.002450,0.000074,0.001712,0.000035,1,{'alpha': 1},0.910448,0.929134,0.951613,0.929134,0.976378,0.939341,0.022652,1
6,0.002348,0.000046,0.001703,0.000041,10,{'alpha': 10},0.916031,0.929134,0.951613,0.930233,0.968750,0.939152,0.018686,2
7,0.002311,0.000035,0.001753,0.000103,100,{'alpha': 100},0.931298,0.921875,0.936508,0.930233,0.953125,0.934608,0.010380,8
8,0.002738,0.000039,0.002084,0.000166,1000,{'alpha': 1000},0.916031,0.890625,0.929134,0.883721,0.951613,0.914225,0.024943,9
9,0.002868,0.000138,0.002042,0.000077,10000,{'alpha': 10000},0.880597,0.888889,0.913386,0.861538,0.943089,0.897500,0.028227,10


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

**** LSVC
0.9357798165137614


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1186: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  raw_coef_, n_iter_ = liblinear.train_wrap(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002349,0.000702,0.002034,0.000246,0.00001,{'C': 1e-05},0.886957,0.854701,0.890756,0.850000,0.933333,0.883149,0.029997,10
1,0.001969,0.000083,0.001819,0.000090,0.0001,{'C': 0.0001},0.870229,0.870968,0.920635,0.873016,0.934426,0.893855,0.027855,9
2,0.001863,0.000069,0.001713,0.000025,0.001,{'C': 0.001},0.930233,0.899225,0.935484,0.899225,0.951613,0.923156,0.020771,6
3,0.001947,0.000070,0.001732,0.000113,0.01,{'C': 0.01},0.931298,0.921875,0.944000,0.930233,0.968750,0.939231,0.016366,5
4,0.004182,0.000594,0.001979,0.000394,0.1,{'C': 0.1},0.930233,0.936508,0.959350,0.930233,0.976378,0.946540,0.018368,1
5,0.020024,0.004627,0.001804,0.000195,1,{'C': 1},0.924242,0.936508,0.959350,0.920635,0.976378,0.943423,0.021329,2
6,0.207144,0.024059,0.002373,0.000668,10,{'C': 10},0.917293,0.936508,0.959350,0.920635,0.976378,0.942033,0.022715,3
7,0.370467,0.028235,0.002593,0.000319,100,{'C': 100},0.917293,0.936508,0.959350,0.920635,0.976378,0.942033,0.022715,3
8,0.378801,0.037965,0.002669,0.000358,1000,{'C': 1000},0.735632,0.936508,0.950820,0.923077,0.976378,0.904483,0.086253,7
9,0.377422,0.040975,0.002291,0.000269,10000,{'C': 10000},0.820513,0.868966,0.944000,0.916031,0.939394,0.897781,0.046908,8
